In [33]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [34]:

# importing basic dataset of the census tracts
schools = pd.read_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/school_census.csv', index_col=0)
census_tracts = pd.read_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/dem_census.csv', index_col=0)

/var/folders/v6/m0s_jr6d723809xzpzrvx_340000gn/T/ipykernel_2428/2463538850.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  census_tracts = pd.read_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/dem_census.csv', index_col=0)


In [35]:
#Turn into geopandas

# This is a helpe function to make grometries work
def wkt_loads(x):
    try:
        return wkt.loads(x)
    except Exception:
        return None
census_tracts["geometry"] = census_tracts['geometry'].apply(wkt_loads)
schools["geometry"] = schools['geometry'].apply(wkt_loads)

geo_dem_census_df = gpd.GeoDataFrame(census_tracts, crs = "EPSG:4674", geometry = 'geometry')
geo_school_census_df = gpd.GeoDataFrame(schools, crs = "EPSG:4674", geometry = 'geometry')
#

/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/geodataframe.py:34: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  out = from_shapely(data)


In [36]:

# Removing the schools and census tracts that we don't have the location of
geo_dem_census_df = geo_dem_census_df[~geo_dem_census_df.geometry.is_empty]
geo_school_census_df = geo_school_census_df[~geo_school_census_df.geometry.is_empty]
geo_dem_census_df = geo_dem_census_df[~geo_dem_census_df.geometry.isna()]
geo_school_census_df = geo_school_census_df[~geo_school_census_df.geometry.isna()]

geo_school_census_df.head()

,region_name,region_id,state_name,state_abbrev,state_id,city_name,city_id,mesoregion_name,mesoregion_id,microregion_name,...,address,address_number,address_complement,neighborhood,zip_code,n_students,n_teachers,n_classes,sigla_uf,geometry
1,Norte,1.0,Rondônia,RO,11.0,Alta Floresta D'Oeste,1100015.0,Leste Rondoniense,1102.0,Cacoal,...,AVENIDA RIO DE JANEIRO,4776,ESCOLA,CIDADE ALTA,76954000.0,0.0,0.0,0.0,RO,POINT (-61.98651 -11.93096)
2,Norte,1.0,Rondônia,RO,11.0,Alta Floresta D'Oeste,1100015.0,Leste Rondoniense,1102.0,Cacoal,...,LINHA 60 COM A 140,NaN,NaN,NaN,76954000.0,0.0,0.0,0.0,RO,POINT (-62.17758 -12.04284)
3,Norte,1.0,Rondônia,RO,11.0,Alta Floresta D'Oeste,1100015.0,Leste Rondoniense,1102.0,Cacoal,...,ROLIM DE MOURA DO GUAPORE,NaN,NaN,NaN,76954000.0,0.0,0.0,0.0,RO,POINT (-62.27693 -13.08367)
7,Norte,1.0,Rondônia,RO,11.0,Alta Floresta D'Oeste,1100015.0,Leste Rondoniense,1102.0,Cacoal,...,LINHA P 50 KM 22,NaN,NaN,NaN,76954000.0,0.0,0.0,0.0,RO,POINT (-61.99335 -12.12982)
8,Norte,1.0,Rondônia,RO,11.0,Alta Floresta D'Oeste,1100015.0,Leste Rondoniense,1102.0,Cacoal,...,AVENIDA MATO GROSSO,3861,NaN,CENTRO,76954000.0,264.0,39.0,27.0,RO,POINT (-61.99506 -11.92733)


In [37]:
# Projecting into the same geometry

# https://epsg.io/5880 
# The units are in meters which facilitates the interpretation of distance
geo_dem_census_df = geo_dem_census_df.to_crs(epsg = 5641)
geo_school_census_df = geo_school_census_df.to_crs(epsg = 5641)


/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out
/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/geoseries.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)


In [38]:
# Only public schools with at least one high school class and regions with at least one high-school aged person
geo_dem_census_df = geo_dem_census_df[geo_dem_census_df["n_people_15to17"] > 0]
geo_school_census_df = geo_school_census_df[(geo_school_census_df["n_classes"] > 0) & (geo_school_census_df["admin_type"] != 4.0)]

In [62]:
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')
def distance_matrix(demand_df, supply_df, name = "euclidean", threshold = 10000):   
    """""
    This function calculates the distance matrix between every pair of points in two geodataframes up to a threshold.

    Inspired by create_euclidean_distances in the Access package
    """
    # Reset the index so that the geoids are accessible
    df1 = demand_df.rename_axis("origin").reset_index()
    df2 = supply_df.rename_axis("dest").reset_index()

    # Convert to centroids if so-specified
    df1.set_geometry(df1.centroid, inplace=True)
    df2.set_geometry(df2.centroid, inplace=True)
    # Calculate the distances.
    if (df1.geom_type == "Point").all() & (df2.geom_type == "Point").all():
        # If both geometries are point types, merge on a temporary dummy column
        df1["temp"] = 1
        df2["temp"] = 1
        df1and2 = df1[["temp", "geometry", "origin"]].merge(
            df2[["temp", "geometry", "dest"]].rename(columns={"geometry": "geomb"})
        )
        df1and2.drop("temp", inplace=True, axis=1)
        df1and2[name] = df1and2.distance(df1and2.set_geometry("geomb"))
    else:
        # Execute an sjoin for non-point geometries, based upon a buffer zone
        df1and2 = gpd.sjoin(
            df1,
            df2.rename(columns={"geometry": "geomb"}).set_geometry(
                df2.buffer(threshold)
            ),
        )
        df1and2[name] = df1and2.distance(df1and2.set_geometry("geomb"))

    # Add it to the cost df.
    df1and2 = df1and2[df1and2[name] < threshold]

    return df1and2


def chunked_distance_matrix(demand_df, supply_df, subset_column, name = "euclidean", threshold = 10000):
    '''''
    This function calculates the distance matrix between every pair of points in two geodataframes up to a threshold.
    It calculates distances separately for each subset to avoid memory issues.
    '''
    subset_values = demand_df[subset_column].unique()
    # Initialize distance matrix df
    distance_matrix_df = pd.DataFrame()
    for value in tqdm(subset_values):
        #print("Processing " + str(value))
        # Subset the dataframes
        demand_subset = demand_df[demand_df[subset_column] == value]
        supply_subset = supply_df[supply_df[subset_column] == value]
        # Calculate the distance matrix
        distance_matrix_df = distance_matrix_df.append(
            distance_matrix(demand_subset, supply_subset, name, threshold)
        )
    return distance_matrix_df

In [63]:
dist_matrix = chunked_distance_matrix(geo_dem_census_df, geo_school_census_df, "city_id", name = "euclidean", threshold = 10000)

100%|██████████| 5565/5565 [09:28<00:00,  9.80it/s]  


In [69]:
dist_matrix = dist_matrix.reset_index()[["origin", "dest", "euclidean"]]

In [70]:
dist_matrix.head()

,origin,dest,euclidean
0,218894,23847,1024.402633
1,268694,23847,704.300224
2,2,23853,1524.518606
3,3,23853,1450.603219
4,4,23853,9231.038217


In [66]:
# Write distance matrix to csv
dist_matrix.to_csv('/Users/feliphlvo/Documents/Minerva/Capstone/data/Local/dist_matrix.csv')

In [45]:
# Unique values of city_id
geo_dem_census_df["city_id"].unique()

array([1700251., 1700301., 1700400., ..., 4207858., 2413805., 2700805.])

In [40]:
# Subset DFs
subset_city = 1100015.0
subset_census = geo_dem_census_df[geo_dem_census_df["city_id"] == subset_city]
subset_schools = geo_school_census_df[geo_school_census_df["city_id"] == subset_city]

In [44]:
distance_matrix(subset_census, subset_schools, name = "euclidean", threshold = 10000)

/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/array.py:546: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return _unary_geo("centroid", self)
/Users/feliphlvo/opt/anaconda3/envs/cap_env/lib/python3.10/site-packages/geopandas/geoseries.py:128: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  data = from_shapely(s.values)


,geometry,origin,geomb,dest,euclidean
0,POINT (2886898.637 8672132.565),10650,POINT (2886759.309 8672190.023),8,150.710340
1,POINT (2886898.637 8672132.565),10650,POINT (2887396.839 8672498.304),12,618.038247
2,POINT (2886898.637 8672132.565),10650,POINT (2885894.346 8671383.156),27,1253.081621
15,POINT (2886629.031 8671143.039),108481,POINT (2886759.309 8672190.023),8,1055.057393
16,POINT (2886629.031 8671143.039),108481,POINT (2887396.839 8672498.304),12,1557.649721
...,...,...,...,...,...
148,POINT (2886438.056 8672714.282),284272,POINT (2887396.839 8672498.304),12,982.807917
149,POINT (2886438.056 8672714.282),284272,POINT (2885894.346 8671383.156),27,1437.886132
156,POINT (2886044.618 8672622.476),306095,POINT (2886759.309 8672190.023),8,835.343847
157,POINT (2886044.618 8672622.476),306095,POINT (2887396.839 8672498.304),12,1357.910520
